<h1>MultiLayerPerceptron Class</h1>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pickle
import matplotlib.patches as mpatches

In [ ]:
class MultiLayerPerceptron:
    def __init__(self, 
                 num_entradas, 
                 num_neuronas_ocultas, 
                 num_salidas, epochs, 
                 inicialitazion_function, 
                 activation_function, 
                 activation_derivative, 
                 loss_function, 
                 normalization_function=None, 
                 batch_size=128, 
                 learning_rate=0.2, 
                 random_state=42):

        self.random_state = random_state
        self.epochs = epochs
        self.batch_size = batch_size
        self.learning_rate = learning_rate
        self.initialize = inicialitazion_function
        self.activate = activation_function
        self.activation_derivative = activation_derivative
        self.loss_function = loss_function
        self.normalize = normalization_function
        
        # Inicializar
        self.seed(self.random_state)
        self.W1 = self.initialize(num_neuronas_ocultas, num_entradas)
        self.B1 = np.zeros((1, num_neuronas_ocultas), dtype=np.float128)
        self.W2 = self.initialize(num_salidas, num_neuronas_ocultas)
        self.B2 = np.zeros((1, num_salidas), dtype=np.float128)

    def seed(self, random_state):
        np.random.seed(random_state)

    def create_minibatches(self, X, y, batch_size):
        """
        Genera los lotes de datos (batchs) de acuerdo al parámetro batch_size de forma aleatoria para el procesamiento. 
        """
        n_samples = X.shape[0]
        indices = np.random.permutation(n_samples)  # Mezcla los índices aleatoriamente
        X_shuffled, y_shuffled = X[indices], y[indices]  # Reordena X e y según los índices aleatorios
        
        # Divide los datos en minibatches
        for X_batch, y_batch in zip(np.array_split(X_shuffled, np.ceil(n_samples / batch_size)), 
                                    np.array_split(y_shuffled, np.ceil(n_samples / batch_size))):
            yield X_batch, y_batch

    def forward(self, X):
        """
        Realiza el Forward pass
        """
        if self.normalize != None:
            self.X = self.normalize(X)
        else:
            self.X = X
        self.z_c1 = self.X @ self.W1.T + self.B1
        self.B1 = np.sum(self.B1, axis=0, keepdims=True)
        self.a_c1 = self.activate(self.z_c1)

        self.z_c2 = self.a_c1 @ self.W2.T + self.B2
        self.B2 = np.sum(self.B2 , axis=0, keepdims=True)
        self.y_pred = self.activate(self.z_c2)
        return self.y_pred 

    def backward(self, y):
        """ 
        Realiza la Backpropagation
        """
        self.dE_dy_pred = self.y_pred - y
        self.dy_pred_dz_c2 = self.activation_derivative(self.y_pred)
        
        self.delta_c2 = self.dE_dy_pred * self.dy_pred_dz_c2
        self.grad_c2 = self.delta_c2.T @ self.a_c1

        self.delta_c1 = self.delta_c2 @ self.W2 * self.activation_derivative(self.a_c1)
        self.grad_c1 = self.delta_c1.T @ self.X

    def update(self):
        """
        Re-ajusta los parámetros de la red
        """
        self.W2 =  self.W2 -  self.learning_rate * self.grad_c2
        self.B2 =  self.B2 - self.learning_rate * np.sum(self.delta_c2, axis=0, keepdims=True)
        
        self.W1 = self.W1 - self.learning_rate * self.grad_c1
        self.B1 = self.B1 - self.learning_rate * np.sum(self.delta_c1, axis=0, keepdims=True)

    def predict(self, X, y):
        """
        Predice como un clasificador binario y establece el acccuracy
        """
        self.y_pred = self.forward(X)
        self.y_pred = np.where(self.y_pred >= 0.5, 1, 0)
    
        errores = np.count_nonzero(self.y_pred - y)
        aciertos = y.shape[0] - errores
        self.accuracy = aciertos / y.shape[0]
        return self.y_pred 

    def train(self, X, Y):
        """
        Entrena a la red neuronal con las features X y el target Y
        1.- Forward Pass
        2.- Cálculo del error
        3.- Backpropagation
        4.- Actualización de parámetros
        """
        self.epochs_error = []
        if self.normalize != None:
            self.X = self.normalize(X)
        else:
            self.X = X
        for epoch in range(self.epochs):
            num_batch = 0
            epoch_error  = 0
            for X_batch, y_batch in self.create_minibatches(X, Y, self.batch_size):
                self.y_pred = self.forward(X_batch)
                error = self.loss_function(self.y_pred, y_batch)
                epoch_error += error    
                self.backward(y_batch)
                self.update()
                num_batch += 1
            self.epochs_error.append(epoch_error/num_batch)
            # Imprimir el error cada N épocas
            if epoch % 100 == 0:
                    print(f"Época {epoch}, Error: {epoch_error/num_batch}")

<h1>Estrategias de ejecución</h1>

In [ ]:
# -----------------------
# Funciones de activación
# -----------------------
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoid_derivative(x):
    return x * (1 - x)

# -----------------------
# Funciones de error
# -----------------------
def loss_function_MSE(y_pred, y):
    error = np.divide((y_pred - y) ** 2, 2 * y.shape[0])
    total_error =  np.sum(error)
    return total_error

# ---------------------------
# Funciones de inicialización
# ---------------------------
def xavier_initialization(input_size, output_size):
    array = np.random.normal(0, np.sqrt(2 / input_size), (input_size, output_size))
    return np.float128(array)
    
def normal_distribution_initialization(input_size, output_size):
    array = np.random.normal(0, 0.1, (input_size, output_size))
    return np.float128(array)

# --------------------------
# Funciones de normalización
# --------------------------
def z_score_normalization(x):
    return (x - np.mean(x)) / np.std(x)

<h1>Leer datos de entrenamiento y prueba</h1>

In [ ]:
# --------------------------
# Compuerta lógica XOR
# --------------------------
X_xor = np.array([[0, 0],
                  [0, 1],
                  [1, 0],
                  [1, 1]])

Y_xor = np.array([[0], 
                  [1], 
                  [1], 
                  [0]])

# --------------------------
# Dataset Iris
# --------------------------
data = np.genfromtxt('./datasets/iris_train.csv', delimiter=',', skip_header=1)
X_iris_train = data[:, :-1]
Y_iris_train = data[:, -1]
Y_iris_train = Y_iris_train.reshape(len(Y_iris_train),1)
data = np.genfromtxt('./datasets/iris_test.csv', delimiter=',', skip_header=1)
X_iris_test = data[:, :-1]
Y_iris_test = data[:, -1]
Y_iris_test = Y_iris_test.reshape(len(Y_iris_test),1)

# --------------------------
# Dataset Breast Cancer
# --------------------------
data = np.genfromtxt('./datasets/breast_cancer_train.csv', delimiter=',', skip_header=1)
X_breast_cancer_train = data[:, :-1]
Y_breast_cancer_train = data[:, -1]
Y_breast_cancer_train = Y_breast_cancer_train.reshape(len(Y_breast_cancer_train),1)
data = np.genfromtxt('./datasets/breast_cancer_test.csv', delimiter=',', skip_header=1)
X_breast_cancer_test = data[:, :-1]
Y_breast_cancer_test = data[:, -1]
Y_breast_cancer_test = Y_breast_cancer_test.reshape(len(Y_breast_cancer_test),1)

# --------------------------
# Dataset Wine
# --------------------------
data = np.genfromtxt('./datasets/wine_train.csv', delimiter=',', skip_header=1)
X_wine_train = data[:, :-1]
Y_wine_train = data[:, -1]
Y_wine_train = Y_wine_train.reshape(len(Y_wine_train),1)
data = np.genfromtxt('./datasets/wine_test.csv', delimiter=',', skip_header=1)
X_wine_test = data[:, :-1]
Y_wine_test = data[:, -1]
Y_wine_test = Y_wine_test.reshape(len(Y_wine_test),1)

<h1>Entrenamiento de los modelos</h1>

In [ ]:
# --------------------------
# Datos de entrenamiento
# --------------------------
X_train = [X_xor, X_iris_train, X_breast_cancer_train, X_wine_train]
Y_train = [Y_xor, Y_iris_train, Y_breast_cancer_train, Y_wine_train]
X_test = [X_xor, X_iris_test, X_breast_cancer_test, X_wine_test]
Y_test = [Y_xor, Y_iris_test, Y_breast_cancer_test, Y_wine_test]
neuronas_ocultas = [2, 4, 8, 16, 32, 128]
salidas = 1
batch_size = [8, 16, 32, 64]
learning_rate = [0.01, 0.1, 0.5]
epochs = 2000
initialization_functions = [xavier_initialization, normal_distribution_initialization]
normalization_functions = [z_score_normalization, None]
nombres_datasets = ['XOR', 'Iris', 'Breast Cancer', 'Wine']
nombres_inicializacion = ['Xavier', 'Normal distribution']
nombres_normalizacion = ['Z-score', 'Ninguna']

In [ ]:
# --------------------------
# Realizar 288 Experimentos por cada Dataset
# --------------------------
lista_clasificadores: list[tuple] = []
for i in range(len(X_train)):
    for i_f_i in range(len(initialization_functions)):
        for i_f_n in range(len(normalization_functions)):
            for n in neuronas_ocultas:
                for lr in learning_rate:
                    for b in batch_size:
                        diccionario = dict()
                        diccionario = {'Dataset': nombres_datasets[i], 
                                    'Initialization':nombres_inicializacion[i_f_i],
                                    'Normalization':nombres_normalizacion[i_f_n],
                                    'Activation':'Sigmoid',
                                    'Input-size':len(X_train[i][0]),
                                    'Output-size':salidas,
                                    'Hidden-layer-size':n,
                                    'Learning-rate':lr,
                                    'Batch-size':b,
                                    'Epochs':epochs,
                                    }
                        print('Entrenando red neuronal con las siguientes características:')
                        print(diccionario)
                        clasificador = MultiLayerPerceptron(num_entradas=len(X_train[i][0]), 
                                                            num_neuronas_ocultas=n, 
                                                            num_salidas=salidas, 
                                                            inicialitazion_function=initialization_functions[i_f_i], 
                                                            normalization_function=normalization_functions[i_f_n],
                                                            loss_function=loss_function_MSE,
                                                            activation_function=sigmoid,
                                                            activation_derivative=sigmoid_derivative,
                                                            epochs=epochs, 
                                                            batch_size=b, 
                                                            learning_rate=lr)
                        clasificador.train(X_train[i], Y_train[i])
                        lista_clasificadores.append((diccionario, clasificador))


<h1>Almacenamiento de los modelos</h1>

In [ ]:
def guardar_modelos(lista_modelos):
    file = open('modelos.pkl', 'wb')    
    pickle.dump(lista_modelos, file)                    
    file.close()  

def cargar_modelos():
    file = open('modelos.pkl', 'rb')    
    lista_modelos = pickle.load(file)
    file.close()
    return lista_modelos

In [ ]:
guardar_modelos(lista_clasificadores)

In [ ]:
lista_clasificadores: list[tuple[dict, MultiLayerPerceptron]] = cargar_modelos()

<h1>Prueba de los modelos</h1>

In [ ]:
mejores_clasificadores = [[], [], [], []]
for d, c in lista_clasificadores:
    i = nombres_datasets.index(d['Dataset'])
    c.predict(X_test[i], Y_test[i])
    d.update({'Accuracy':c.accuracy, 'Last-error':c.epochs_error[-1]})
    if len(mejores_clasificadores[i]) != 0:
        if mejores_clasificadores[i][0][0]['Accuracy'] == d['Accuracy']:
            mejores_clasificadores[i].append((d, c))
        elif mejores_clasificadores[i][0][0]['Accuracy'] < d['Accuracy']:
            mejores_clasificadores[i] = [(d, c)]
    else:
        mejores_clasificadores[i] = [(d, c)]  

In [ ]:
for lista in mejores_clasificadores:
    print(f'\n#######Dataset {lista[0][0]['Dataset']}#######\n')
    print(f'Mejor accuracy: {lista[0][0]['Accuracy']}')
    if len(lista) > 1:
        print(f'Hay {len(lista)} clasificadores con el mismo accuracy')
        for d, c in lista:
            print(d)
    else:
        print(lista[0][0])

<h1>Lista de los mejores modelos</h1>
<h2>Dataset XOR</h2>
<h3>Mejor accuracy: 1.0<br/>
Hay 112 clasificadores con el mismo accuracy</h3>
<font size='2'>
<ul>
<li>{'Dataset': 'XOR', 'Initialization': 'Xavier', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 2, 'Output-size': 1, 'Hidden-layer-size': 2, 'Learning-rate': 0.5, 'Batch-size': 8, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.0009244202344850484718')}</li>
<li>{'Dataset': 'XOR', 'Initialization': 'Xavier', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 2, 'Output-size': 1, 'Hidden-layer-size': 2, 'Learning-rate': 0.5, 'Batch-size': 16, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.0009244202344850484718')}</li>
<li>{'Dataset': 'XOR', 'Initialization': 'Xavier', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 2, 'Output-size': 1, 'Hidden-layer-size': 2, 'Learning-rate': 0.5, 'Batch-size': 32, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.0009244202344850484718')}</li>
<li>{'Dataset': 'XOR', 'Initialization': 'Xavier', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 2, 'Output-size': 1, 'Hidden-layer-size': 2, 'Learning-rate': 0.5, 'Batch-size': 64, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.0009244202344850484718')}</li>
<li>{'Dataset': 'XOR', 'Initialization': 'Xavier', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 2, 'Output-size': 1, 'Hidden-layer-size': 4, 'Learning-rate': 0.1, 'Batch-size': 8, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.006825906507797358372')}</li>
<li>{'Dataset': 'XOR', 'Initialization': 'Xavier', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 2, 'Output-size': 1, 'Hidden-layer-size': 4, 'Learning-rate': 0.1, 'Batch-size': 16, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.006825906507797358372')}</li>
<li>{'Dataset': 'XOR', 'Initialization': 'Xavier', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 2, 'Output-size': 1, 'Hidden-layer-size': 4, 'Learning-rate': 0.1, 'Batch-size': 32, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.006825906507797358372')}</li>
<li>{'Dataset': 'XOR', 'Initialization': 'Xavier', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 2, 'Output-size': 1, 'Hidden-layer-size': 4, 'Learning-rate': 0.1, 'Batch-size': 64, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.006825906507797358372')}</li>
<li>{'Dataset': 'XOR', 'Initialization': 'Xavier', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 2, 'Output-size': 1, 'Hidden-layer-size': 4, 'Learning-rate': 0.5, 'Batch-size': 8, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.0007034682531777309165')}</li>
<li>{'Dataset': 'XOR', 'Initialization': 'Xavier', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 2, 'Output-size': 1, 'Hidden-layer-size': 4, 'Learning-rate': 0.5, 'Batch-size': 16, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.0007034682531777309165')}</li>
<li>{'Dataset': 'XOR', 'Initialization': 'Xavier', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 2, 'Output-size': 1, 'Hidden-layer-size': 4, 'Learning-rate': 0.5, 'Batch-size': 32, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.0007034682531777309165')}</li>
<li>{'Dataset': 'XOR', 'Initialization': 'Xavier', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 2, 'Output-size': 1, 'Hidden-layer-size': 4, 'Learning-rate': 0.5, 'Batch-size': 64, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.0007034682531777309165')}</li>
<li>{'Dataset': 'XOR', 'Initialization': 'Xavier', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 2, 'Output-size': 1, 'Hidden-layer-size': 8, 'Learning-rate': 0.01, 'Batch-size': 8, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.10404053740422085827')}</li>
<li>{'Dataset': 'XOR', 'Initialization': 'Xavier', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 2, 'Output-size': 1, 'Hidden-layer-size': 8, 'Learning-rate': 0.01, 'Batch-size': 16, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.10404053740422085827')}</li>
<li>{'Dataset': 'XOR', 'Initialization': 'Xavier', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 2, 'Output-size': 1, 'Hidden-layer-size': 8, 'Learning-rate': 0.01, 'Batch-size': 32, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.10404053740422085827')}</li>
<li>{'Dataset': 'XOR', 'Initialization': 'Xavier', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 2, 'Output-size': 1, 'Hidden-layer-size': 8, 'Learning-rate': 0.01, 'Batch-size': 64, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.10404053740422085827')}</li>
<li>{'Dataset': 'XOR', 'Initialization': 'Xavier', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 2, 'Output-size': 1, 'Hidden-layer-size': 8, 'Learning-rate': 0.1, 'Batch-size': 8, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.004341217752180242664')}</li>
<li>{'Dataset': 'XOR', 'Initialization': 'Xavier', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 2, 'Output-size': 1, 'Hidden-layer-size': 8, 'Learning-rate': 0.1, 'Batch-size': 16, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.004341217752180242664')}</li>
<li>{'Dataset': 'XOR', 'Initialization': 'Xavier', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 2, 'Output-size': 1, 'Hidden-layer-size': 8, 'Learning-rate': 0.1, 'Batch-size': 32, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.004341217752180242664')}</li>
<li>{'Dataset': 'XOR', 'Initialization': 'Xavier', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 2, 'Output-size': 1, 'Hidden-layer-size': 8, 'Learning-rate': 0.1, 'Batch-size': 64, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.004341217752180242664')}</li>
<li>{'Dataset': 'XOR', 'Initialization': 'Xavier', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 2, 'Output-size': 1, 'Hidden-layer-size': 8, 'Learning-rate': 0.5, 'Batch-size': 8, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.0004895566860031729911')}</li>
<li>{'Dataset': 'XOR', 'Initialization': 'Xavier', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 2, 'Output-size': 1, 'Hidden-layer-size': 8, 'Learning-rate': 0.5, 'Batch-size': 16, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.0004895566860031729911')}</li>
<li>{'Dataset': 'XOR', 'Initialization': 'Xavier', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 2, 'Output-size': 1, 'Hidden-layer-size': 8, 'Learning-rate': 0.5, 'Batch-size': 32, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.0004895566860031729911')}</li>
<li>{'Dataset': 'XOR', 'Initialization': 'Xavier', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 2, 'Output-size': 1, 'Hidden-layer-size': 8, 'Learning-rate': 0.5, 'Batch-size': 64, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.0004895566860031729911')}</li>
<li>{'Dataset': 'XOR', 'Initialization': 'Xavier', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 2, 'Output-size': 1, 'Hidden-layer-size': 16, 'Learning-rate': 0.1, 'Batch-size': 8, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.0033317845688289904504')}</li>
<li>{'Dataset': 'XOR', 'Initialization': 'Xavier', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 2, 'Output-size': 1, 'Hidden-layer-size': 16, 'Learning-rate': 0.1, 'Batch-size': 16, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.0033317845688289904504')}</li>
<li>{'Dataset': 'XOR', 'Initialization': 'Xavier', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 2, 'Output-size': 1, 'Hidden-layer-size': 16, 'Learning-rate': 0.1, 'Batch-size': 32, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.0033317845688289904504')}</li>
<li>{'Dataset': 'XOR', 'Initialization': 'Xavier', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 2, 'Output-size': 1, 'Hidden-layer-size': 16, 'Learning-rate': 0.1, 'Batch-size': 64, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.0033317845688289904504')}</li>
<li>{'Dataset': 'XOR', 'Initialization': 'Xavier', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 2, 'Output-size': 1, 'Hidden-layer-size': 16, 'Learning-rate': 0.5, 'Batch-size': 8, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.00034108285839430428633')}</li>
<li>{'Dataset': 'XOR', 'Initialization': 'Xavier', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 2, 'Output-size': 1, 'Hidden-layer-size': 16, 'Learning-rate': 0.5, 'Batch-size': 16, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.00034108285839430428633')}</li>
<li>{'Dataset': 'XOR', 'Initialization': 'Xavier', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 2, 'Output-size': 1, 'Hidden-layer-size': 16, 'Learning-rate': 0.5, 'Batch-size': 32, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.00034108285839430428633')}</li>
<li>{'Dataset': 'XOR', 'Initialization': 'Xavier', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 2, 'Output-size': 1, 'Hidden-layer-size': 16, 'Learning-rate': 0.5, 'Batch-size': 64, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.00034108285839430428633')}</li>
<li>{'Dataset': 'XOR', 'Initialization': 'Xavier', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 2, 'Output-size': 1, 'Hidden-layer-size': 32, 'Learning-rate': 0.01, 'Batch-size': 8, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.11143629462790004088')}</li>
<li>{'Dataset': 'XOR', 'Initialization': 'Xavier', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 2, 'Output-size': 1, 'Hidden-layer-size': 32, 'Learning-rate': 0.01, 'Batch-size': 16, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.11143629462790004088')}</li>
<li>{'Dataset': 'XOR', 'Initialization': 'Xavier', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 2, 'Output-size': 1, 'Hidden-layer-size': 32, 'Learning-rate': 0.01, 'Batch-size': 32, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.11143629462790004088')}</li>
<li>{'Dataset': 'XOR', 'Initialization': 'Xavier', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 2, 'Output-size': 1, 'Hidden-layer-size': 32, 'Learning-rate': 0.01, 'Batch-size': 64, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.11143629462790004088')}</li>
<li>{'Dataset': 'XOR', 'Initialization': 'Xavier', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 2, 'Output-size': 1, 'Hidden-layer-size': 32, 'Learning-rate': 0.1, 'Batch-size': 8, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.0020832410594029358554')}</li>
<li>{'Dataset': 'XOR', 'Initialization': 'Xavier', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 2, 'Output-size': 1, 'Hidden-layer-size': 32, 'Learning-rate': 0.1, 'Batch-size': 16, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.0020832410594029358554')}</li>
<li>{'Dataset': 'XOR', 'Initialization': 'Xavier', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 2, 'Output-size': 1, 'Hidden-layer-size': 32, 'Learning-rate': 0.1, 'Batch-size': 32, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.0020832410594029358554')}</li>
<li>{'Dataset': 'XOR', 'Initialization': 'Xavier', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 2, 'Output-size': 1, 'Hidden-layer-size': 32, 'Learning-rate': 0.1, 'Batch-size': 64, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.0020832410594029358554')}</li>
<li>{'Dataset': 'XOR', 'Initialization': 'Xavier', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 2, 'Output-size': 1, 'Hidden-layer-size': 32, 'Learning-rate': 0.5, 'Batch-size': 8, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.00023765149326017377023')}</li>
<li>{'Dataset': 'XOR', 'Initialization': 'Xavier', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 2, 'Output-size': 1, 'Hidden-layer-size': 32, 'Learning-rate': 0.5, 'Batch-size': 16, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.00023765149326017377023')}</li>
<li>{'Dataset': 'XOR', 'Initialization': 'Xavier', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 2, 'Output-size': 1, 'Hidden-layer-size': 32, 'Learning-rate': 0.5, 'Batch-size': 32, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.00023765149326017377023')}</li>
<li>{'Dataset': 'XOR', 'Initialization': 'Xavier', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 2, 'Output-size': 1, 'Hidden-layer-size': 32, 'Learning-rate': 0.5, 'Batch-size': 64, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.00023765149326017377023')}</li>
<li>{'Dataset': 'XOR', 'Initialization': 'Xavier', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 2, 'Output-size': 1, 'Hidden-layer-size': 128, 'Learning-rate': 0.01, 'Batch-size': 8, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.124239476944594147924')}</li>
<li>{'Dataset': 'XOR', 'Initialization': 'Xavier', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 2, 'Output-size': 1, 'Hidden-layer-size': 128, 'Learning-rate': 0.01, 'Batch-size': 16, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.124239476944594147924')}</li>
<li>{'Dataset': 'XOR', 'Initialization': 'Xavier', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 2, 'Output-size': 1, 'Hidden-layer-size': 128, 'Learning-rate': 0.01, 'Batch-size': 32, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.124239476944594147924')}</li>
<li>{'Dataset': 'XOR', 'Initialization': 'Xavier', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 2, 'Output-size': 1, 'Hidden-layer-size': 128, 'Learning-rate': 0.01, 'Batch-size': 64, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.124239476944594147924')}</li>
<li>{'Dataset': 'XOR', 'Initialization': 'Xavier', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 2, 'Output-size': 1, 'Hidden-layer-size': 128, 'Learning-rate': 0.1, 'Batch-size': 8, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.0021607904195374907425')}</li>
<li>{'Dataset': 'XOR', 'Initialization': 'Xavier', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 2, 'Output-size': 1, 'Hidden-layer-size': 128, 'Learning-rate': 0.1, 'Batch-size': 16, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.0021607904195374907425')}</li>
<li>{'Dataset': 'XOR', 'Initialization': 'Xavier', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 2, 'Output-size': 1, 'Hidden-layer-size': 128, 'Learning-rate': 0.1, 'Batch-size': 32, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.0021607904195374907425')}</li>
<li>{'Dataset': 'XOR', 'Initialization': 'Xavier', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 2, 'Output-size': 1, 'Hidden-layer-size': 128, 'Learning-rate': 0.1, 'Batch-size': 64, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.0021607904195374907425')}</li>
<li>{'Dataset': 'XOR', 'Initialization': 'Xavier', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 2, 'Output-size': 1, 'Hidden-layer-size': 128, 'Learning-rate': 0.5, 'Batch-size': 8, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.00021421495922037760613')}</li>
<li>{'Dataset': 'XOR', 'Initialization': 'Xavier', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 2, 'Output-size': 1, 'Hidden-layer-size': 128, 'Learning-rate': 0.5, 'Batch-size': 16, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.00021421495922037760613')}</li>
<li>{'Dataset': 'XOR', 'Initialization': 'Xavier', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 2, 'Output-size': 1, 'Hidden-layer-size': 128, 'Learning-rate': 0.5, 'Batch-size': 32, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.00021421495922037760613')}</li>
<li>{'Dataset': 'XOR', 'Initialization': 'Xavier', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 2, 'Output-size': 1, 'Hidden-layer-size': 128, 'Learning-rate': 0.5, 'Batch-size': 64, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.00021421495922037760613')}</li>
<li>{'Dataset': 'XOR', 'Initialization': 'Xavier', 'Normalization': 'Ninguna', 'Activation': 'Sigmoid', 'Input-size': 2, 'Output-size': 1, 'Hidden-layer-size': 2, 'Learning-rate': 0.5, 'Batch-size': 8, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.0013673019683460498371')}</li>
<li>{'Dataset': 'XOR', 'Initialization': 'Xavier', 'Normalization': 'Ninguna', 'Activation': 'Sigmoid', 'Input-size': 2, 'Output-size': 1, 'Hidden-layer-size': 2, 'Learning-rate': 0.5, 'Batch-size': 16, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.0013673019683460498371')}</li>
<li>{'Dataset': 'XOR', 'Initialization': 'Xavier', 'Normalization': 'Ninguna', 'Activation': 'Sigmoid', 'Input-size': 2, 'Output-size': 1, 'Hidden-layer-size': 2, 'Learning-rate': 0.5, 'Batch-size': 32, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.0013673019683460498371')}</li>
<li>{'Dataset': 'XOR', 'Initialization': 'Xavier', 'Normalization': 'Ninguna', 'Activation': 'Sigmoid', 'Input-size': 2, 'Output-size': 1, 'Hidden-layer-size': 2, 'Learning-rate': 0.5, 'Batch-size': 64, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.0013673019683460498371')}</li>
<li>{'Dataset': 'XOR', 'Initialization': 'Xavier', 'Normalization': 'Ninguna', 'Activation': 'Sigmoid', 'Input-size': 2, 'Output-size': 1, 'Hidden-layer-size': 4, 'Learning-rate': 0.1, 'Batch-size': 8, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.06950847859492007098')}</li>
<li>{'Dataset': 'XOR', 'Initialization': 'Xavier', 'Normalization': 'Ninguna', 'Activation': 'Sigmoid', 'Input-size': 2, 'Output-size': 1, 'Hidden-layer-size': 4, 'Learning-rate': 0.1, 'Batch-size': 16, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.06950847859492007098')}</li>
<li>{'Dataset': 'XOR', 'Initialization': 'Xavier', 'Normalization': 'Ninguna', 'Activation': 'Sigmoid', 'Input-size': 2, 'Output-size': 1, 'Hidden-layer-size': 4, 'Learning-rate': 0.1, 'Batch-size': 32, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.06950847859492007098')}</li>
<li>{'Dataset': 'XOR', 'Initialization': 'Xavier', 'Normalization': 'Ninguna', 'Activation': 'Sigmoid', 'Input-size': 2, 'Output-size': 1, 'Hidden-layer-size': 4, 'Learning-rate': 0.1, 'Batch-size': 64, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.06950847859492007098')}</li>
<li>{'Dataset': 'XOR', 'Initialization': 'Xavier', 'Normalization': 'Ninguna', 'Activation': 'Sigmoid', 'Input-size': 2, 'Output-size': 1, 'Hidden-layer-size': 4, 'Learning-rate': 0.5, 'Batch-size': 8, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.0011679517903124673413')}</li>
<li>{'Dataset': 'XOR', 'Initialization': 'Xavier', 'Normalization': 'Ninguna', 'Activation': 'Sigmoid', 'Input-size': 2, 'Output-size': 1, 'Hidden-layer-size': 4, 'Learning-rate': 0.5, 'Batch-size': 16, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.0011679517903124673413')}</li>
<li>{'Dataset': 'XOR', 'Initialization': 'Xavier', 'Normalization': 'Ninguna', 'Activation': 'Sigmoid', 'Input-size': 2, 'Output-size': 1, 'Hidden-layer-size': 4, 'Learning-rate': 0.5, 'Batch-size': 32, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.0011679517903124673413')}</li>
<li>{'Dataset': 'XOR', 'Initialization': 'Xavier', 'Normalization': 'Ninguna', 'Activation': 'Sigmoid', 'Input-size': 2, 'Output-size': 1, 'Hidden-layer-size': 4, 'Learning-rate': 0.5, 'Batch-size': 64, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.0011679517903124673413')}</li>
<li>{'Dataset': 'XOR', 'Initialization': 'Xavier', 'Normalization': 'Ninguna', 'Activation': 'Sigmoid', 'Input-size': 2, 'Output-size': 1, 'Hidden-layer-size': 8, 'Learning-rate': 0.1, 'Batch-size': 8, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.025444494498382597797')}</li>
<li>{'Dataset': 'XOR', 'Initialization': 'Xavier', 'Normalization': 'Ninguna', 'Activation': 'Sigmoid', 'Input-size': 2, 'Output-size': 1, 'Hidden-layer-size': 8, 'Learning-rate': 0.1, 'Batch-size': 16, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.025444494498382597797')}</li>
<li>{'Dataset': 'XOR', 'Initialization': 'Xavier', 'Normalization': 'Ninguna', 'Activation': 'Sigmoid', 'Input-size': 2, 'Output-size': 1, 'Hidden-layer-size': 8, 'Learning-rate': 0.1, 'Batch-size': 32, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.025444494498382597797')}</li>
<li>{'Dataset': 'XOR', 'Initialization': 'Xavier', 'Normalization': 'Ninguna', 'Activation': 'Sigmoid', 'Input-size': 2, 'Output-size': 1, 'Hidden-layer-size': 8, 'Learning-rate': 0.1, 'Batch-size': 64, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.025444494498382597797')}</li>
<li>{'Dataset': 'XOR', 'Initialization': 'Xavier', 'Normalization': 'Ninguna', 'Activation': 'Sigmoid', 'Input-size': 2, 'Output-size': 1, 'Hidden-layer-size': 8, 'Learning-rate': 0.5, 'Batch-size': 8, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.0008213094036738231356')}</li>
<li>{'Dataset': 'XOR', 'Initialization': 'Xavier', 'Normalization': 'Ninguna', 'Activation': 'Sigmoid', 'Input-size': 2, 'Output-size': 1, 'Hidden-layer-size': 8, 'Learning-rate': 0.5, 'Batch-size': 16, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.0008213094036738231356')}</li>
<li>{'Dataset': 'XOR', 'Initialization': 'Xavier', 'Normalization': 'Ninguna', 'Activation': 'Sigmoid', 'Input-size': 2, 'Output-size': 1, 'Hidden-layer-size': 8, 'Learning-rate': 0.5, 'Batch-size': 32, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.0008213094036738231356')}</li>
<li>{'Dataset': 'XOR', 'Initialization': 'Xavier', 'Normalization': 'Ninguna', 'Activation': 'Sigmoid', 'Input-size': 2, 'Output-size': 1, 'Hidden-layer-size': 8, 'Learning-rate': 0.5, 'Batch-size': 64, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.0008213094036738231356')}</li>
<li>{'Dataset': 'XOR', 'Initialization': 'Xavier', 'Normalization': 'Ninguna', 'Activation': 'Sigmoid', 'Input-size': 2, 'Output-size': 1, 'Hidden-layer-size': 16, 'Learning-rate': 0.1, 'Batch-size': 8, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.032181302878570813728')}</li>
<li>{'Dataset': 'XOR', 'Initialization': 'Xavier', 'Normalization': 'Ninguna', 'Activation': 'Sigmoid', 'Input-size': 2, 'Output-size': 1, 'Hidden-layer-size': 16, 'Learning-rate': 0.1, 'Batch-size': 16, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.032181302878570813728')}</li>
<li>{'Dataset': 'XOR', 'Initialization': 'Xavier', 'Normalization': 'Ninguna', 'Activation': 'Sigmoid', 'Input-size': 2, 'Output-size': 1, 'Hidden-layer-size': 16, 'Learning-rate': 0.1, 'Batch-size': 32, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.032181302878570813728')}</li>
<li>{'Dataset': 'XOR', 'Initialization': 'Xavier', 'Normalization': 'Ninguna', 'Activation': 'Sigmoid', 'Input-size': 2, 'Output-size': 1, 'Hidden-layer-size': 16, 'Learning-rate': 0.1, 'Batch-size': 64, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.032181302878570813728')}</li>
<li>{'Dataset': 'XOR', 'Initialization': 'Xavier', 'Normalization': 'Ninguna', 'Activation': 'Sigmoid', 'Input-size': 2, 'Output-size': 1, 'Hidden-layer-size': 16, 'Learning-rate': 0.5, 'Batch-size': 8, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.00082412234043520785235')}</li>
<li>{'Dataset': 'XOR', 'Initialization': 'Xavier', 'Normalization': 'Ninguna', 'Activation': 'Sigmoid', 'Input-size': 2, 'Output-size': 1, 'Hidden-layer-size': 16, 'Learning-rate': 0.5, 'Batch-size': 16, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.00082412234043520785235')}</li>
<li>{'Dataset': 'XOR', 'Initialization': 'Xavier', 'Normalization': 'Ninguna', 'Activation': 'Sigmoid', 'Input-size': 2, 'Output-size': 1, 'Hidden-layer-size': 16, 'Learning-rate': 0.5, 'Batch-size': 32, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.00082412234043520785235')}</li>
<li>{'Dataset': 'XOR', 'Initialization': 'Xavier', 'Normalization': 'Ninguna', 'Activation': 'Sigmoid', 'Input-size': 2, 'Output-size': 1, 'Hidden-layer-size': 16, 'Learning-rate': 0.5, 'Batch-size': 64, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.00082412234043520785235')}</li>
<li>{'Dataset': 'XOR', 'Initialization': 'Xavier', 'Normalization': 'Ninguna', 'Activation': 'Sigmoid', 'Input-size': 2, 'Output-size': 1, 'Hidden-layer-size': 32, 'Learning-rate': 0.1, 'Batch-size': 8, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.008706357984390136641')}</li>
<li>{'Dataset': 'XOR', 'Initialization': 'Xavier', 'Normalization': 'Ninguna', 'Activation': 'Sigmoid', 'Input-size': 2, 'Output-size': 1, 'Hidden-layer-size': 32, 'Learning-rate': 0.1, 'Batch-size': 16, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.008706357984390136641')}</li>
<li>{'Dataset': 'XOR', 'Initialization': 'Xavier', 'Normalization': 'Ninguna', 'Activation': 'Sigmoid', 'Input-size': 2, 'Output-size': 1, 'Hidden-layer-size': 32, 'Learning-rate': 0.1, 'Batch-size': 32, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.008706357984390136641')}</li>
<li>{'Dataset': 'XOR', 'Initialization': 'Xavier', 'Normalization': 'Ninguna', 'Activation': 'Sigmoid', 'Input-size': 2, 'Output-size': 1, 'Hidden-layer-size': 32, 'Learning-rate': 0.1, 'Batch-size': 64, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.008706357984390136641')}</li>
<li>{'Dataset': 'XOR', 'Initialization': 'Xavier', 'Normalization': 'Ninguna', 'Activation': 'Sigmoid', 'Input-size': 2, 'Output-size': 1, 'Hidden-layer-size': 32, 'Learning-rate': 0.5, 'Batch-size': 8, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.000574694410653426024')}</li>
<li>{'Dataset': 'XOR', 'Initialization': 'Xavier', 'Normalization': 'Ninguna', 'Activation': 'Sigmoid', 'Input-size': 2, 'Output-size': 1, 'Hidden-layer-size': 32, 'Learning-rate': 0.5, 'Batch-size': 16, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.000574694410653426024')}</li>
<li>{'Dataset': 'XOR', 'Initialization': 'Xavier', 'Normalization': 'Ninguna', 'Activation': 'Sigmoid', 'Input-size': 2, 'Output-size': 1, 'Hidden-layer-size': 32, 'Learning-rate': 0.5, 'Batch-size': 32, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.000574694410653426024')}</li>
<li>{'Dataset': 'XOR', 'Initialization': 'Xavier', 'Normalization': 'Ninguna', 'Activation': 'Sigmoid', 'Input-size': 2, 'Output-size': 1, 'Hidden-layer-size': 32, 'Learning-rate': 0.5, 'Batch-size': 64, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.000574694410653426024')}</li>
<li>{'Dataset': 'XOR', 'Initialization': 'Xavier', 'Normalization': 'Ninguna', 'Activation': 'Sigmoid', 'Input-size': 2, 'Output-size': 1, 'Hidden-layer-size': 128, 'Learning-rate': 0.1, 'Batch-size': 8, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.05204906288875691528')}</li>
<li>{'Dataset': 'XOR', 'Initialization': 'Xavier', 'Normalization': 'Ninguna', 'Activation': 'Sigmoid', 'Input-size': 2, 'Output-size': 1, 'Hidden-layer-size': 128, 'Learning-rate': 0.1, 'Batch-size': 16, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.05204906288875691528')}</li>
<li>{'Dataset': 'XOR', 'Initialization': 'Xavier', 'Normalization': 'Ninguna', 'Activation': 'Sigmoid', 'Input-size': 2, 'Output-size': 1, 'Hidden-layer-size': 128, 'Learning-rate': 0.1, 'Batch-size': 32, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.05204906288875691528')}</li>
<li>{'Dataset': 'XOR', 'Initialization': 'Xavier', 'Normalization': 'Ninguna', 'Activation': 'Sigmoid', 'Input-size': 2, 'Output-size': 1, 'Hidden-layer-size': 128, 'Learning-rate': 0.1, 'Batch-size': 64, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.05204906288875691528')}</li>
<li>{'Dataset': 'XOR', 'Initialization': 'Xavier', 'Normalization': 'Ninguna', 'Activation': 'Sigmoid', 'Input-size': 2, 'Output-size': 1, 'Hidden-layer-size': 128, 'Learning-rate': 0.5, 'Batch-size': 8, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.0010002588733707364752')}</li>
<li>{'Dataset': 'XOR', 'Initialization': 'Xavier', 'Normalization': 'Ninguna', 'Activation': 'Sigmoid', 'Input-size': 2, 'Output-size': 1, 'Hidden-layer-size': 128, 'Learning-rate': 0.5, 'Batch-size': 16, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.0010002588733707364752')}</li>
<li>{'Dataset': 'XOR', 'Initialization': 'Xavier', 'Normalization': 'Ninguna', 'Activation': 'Sigmoid', 'Input-size': 2, 'Output-size': 1, 'Hidden-layer-size': 128, 'Learning-rate': 0.5, 'Batch-size': 32, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.0010002588733707364752')}</li>
<li>{'Dataset': 'XOR', 'Initialization': 'Xavier', 'Normalization': 'Ninguna', 'Activation': 'Sigmoid', 'Input-size': 2, 'Output-size': 1, 'Hidden-layer-size': 128, 'Learning-rate': 0.5, 'Batch-size': 64, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.0010002588733707364752')}</li>
<li>{'Dataset': 'XOR', 'Initialization': 'Normal distribution', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 2, 'Output-size': 1, 'Hidden-layer-size': 32, 'Learning-rate': 0.5, 'Batch-size': 8, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.07455191003795318361')}</li>
<li>{'Dataset': 'XOR', 'Initialization': 'Normal distribution', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 2, 'Output-size': 1, 'Hidden-layer-size': 32, 'Learning-rate': 0.5, 'Batch-size': 16, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.07455191003795318361')}</li>
<li>{'Dataset': 'XOR', 'Initialization': 'Normal distribution', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 2, 'Output-size': 1, 'Hidden-layer-size': 32, 'Learning-rate': 0.5, 'Batch-size': 32, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.07455191003795318361')}</li>
<li>{'Dataset': 'XOR', 'Initialization': 'Normal distribution', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 2, 'Output-size': 1, 'Hidden-layer-size': 32, 'Learning-rate': 0.5, 'Batch-size': 64, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.07455191003795318361')}</li>
<li>{'Dataset': 'XOR', 'Initialization': 'Normal distribution', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 2, 'Output-size': 1, 'Hidden-layer-size': 128, 'Learning-rate': 0.5, 'Batch-size': 8, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.0008669379356473800426')}</li>
<li>{'Dataset': 'XOR', 'Initialization': 'Normal distribution', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 2, 'Output-size': 1, 'Hidden-layer-size': 128, 'Learning-rate': 0.5, 'Batch-size': 16, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.0008669379356473800426')}</li>
<li>{'Dataset': 'XOR', 'Initialization': 'Normal distribution', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 2, 'Output-size': 1, 'Hidden-layer-size': 128, 'Learning-rate': 0.5, 'Batch-size': 32, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.0008669379356473800426')}</li>
<li>{'Dataset': 'XOR', 'Initialization': 'Normal distribution', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 2, 'Output-size': 1, 'Hidden-layer-size': 128, 'Learning-rate': 0.5, 'Batch-size': 64, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.0008669379356473800426')}</li>
<li>{'Dataset': 'XOR', 'Initialization': 'Normal distribution', 'Normalization': 'Ninguna', 'Activation': 'Sigmoid', 'Input-size': 2, 'Output-size': 1, 'Hidden-layer-size': 128, 'Learning-rate': 0.5, 'Batch-size': 8, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.021541853173762580541')}</li>
<li>{'Dataset': 'XOR', 'Initialization': 'Normal distribution', 'Normalization': 'Ninguna', 'Activation': 'Sigmoid', 'Input-size': 2, 'Output-size': 1, 'Hidden-layer-size': 128, 'Learning-rate': 0.5, 'Batch-size': 16, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.021541853173762580541')}</li>
<li>{'Dataset': 'XOR', 'Initialization': 'Normal distribution', 'Normalization': 'Ninguna', 'Activation': 'Sigmoid', 'Input-size': 2, 'Output-size': 1, 'Hidden-layer-size': 128, 'Learning-rate': 0.5, 'Batch-size': 32, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.021541853173762580541')}</li>
<li>{'Dataset': 'XOR', 'Initialization': 'Normal distribution', 'Normalization': 'Ninguna', 'Activation': 'Sigmoid', 'Input-size': 2, 'Output-size': 1, 'Hidden-layer-size': 128, 'Learning-rate': 0.5, 'Batch-size': 64, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.021541853173762580541')}</li>
</ul>
</font>
<h2>Dataset Iris</h2>
<h3>Mejor accuracy: 1.0<br/>
Hay 264 clasificadores con el mismo accuracy</h3>
<font size='2'>
<ul>
<li>{'Dataset': 'Iris', 'Initialization': 'Xavier', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 2, 'Learning-rate': 0.01, 'Batch-size': 8, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.0015620343649324747202')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Xavier', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 2, 'Learning-rate': 0.01, 'Batch-size': 16, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.0014542251250858597148')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Xavier', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 2, 'Learning-rate': 0.01, 'Batch-size': 32, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.0010222698162968535253')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Xavier', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 2, 'Learning-rate': 0.01, 'Batch-size': 64, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.00097380162210501163974')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Xavier', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 2, 'Learning-rate': 0.1, 'Batch-size': 8, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.00013308102672031491983')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Xavier', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 2, 'Learning-rate': 0.1, 'Batch-size': 16, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.00013117489204502800124')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Xavier', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 2, 'Learning-rate': 0.1, 'Batch-size': 32, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('7.4582362434891161765e-05')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Xavier', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 2, 'Learning-rate': 0.1, 'Batch-size': 64, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('6.9719020250499072716e-05')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Xavier', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 2, 'Learning-rate': 0.5, 'Batch-size': 8, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('2.3722269814849591713e-05')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Xavier', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 2, 'Learning-rate': 0.5, 'Batch-size': 16, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('2.4422459418975833616e-05')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Xavier', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 2, 'Learning-rate': 0.5, 'Batch-size': 32, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('1.3184570196125529519e-05')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Xavier', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 2, 'Learning-rate': 0.5, 'Batch-size': 64, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('1.2348742182197843992e-05')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Xavier', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 4, 'Learning-rate': 0.01, 'Batch-size': 8, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.0011808076715760034978')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Xavier', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 4, 'Learning-rate': 0.01, 'Batch-size': 16, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.0010618145254809556923')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Xavier', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 4, 'Learning-rate': 0.01, 'Batch-size': 32, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.0005355704354314458432')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Xavier', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 4, 'Learning-rate': 0.01, 'Batch-size': 64, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.00047402465689468105767')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Xavier', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 4, 'Learning-rate': 0.1, 'Batch-size': 8, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.00010755856680177347947')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Xavier', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 4, 'Learning-rate': 0.1, 'Batch-size': 16, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.000103285233135539663016')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Xavier', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 4, 'Learning-rate': 0.1, 'Batch-size': 32, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('4.0932450407487614685e-05')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Xavier', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 4, 'Learning-rate': 0.1, 'Batch-size': 64, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('3.46687316634502521e-05')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Xavier', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 4, 'Learning-rate': 0.5, 'Batch-size': 8, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('1.8117283655223987212e-05')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Xavier', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 4, 'Learning-rate': 0.5, 'Batch-size': 16, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('1.8567312885815924577e-05')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Xavier', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 4, 'Learning-rate': 0.5, 'Batch-size': 32, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('7.1187526348988297847e-06')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Xavier', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 4, 'Learning-rate': 0.5, 'Batch-size': 64, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('6.022127667030645293e-06')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Xavier', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 8, 'Learning-rate': 0.01, 'Batch-size': 8, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.00021920722853233850606')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Xavier', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 8, 'Learning-rate': 0.01, 'Batch-size': 16, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.00027561280296697018163')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Xavier', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 8, 'Learning-rate': 0.01, 'Batch-size': 32, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.0002810633849895717512')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Xavier', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 8, 'Learning-rate': 0.01, 'Batch-size': 64, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.0003307771715813671592')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Xavier', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 8, 'Learning-rate': 0.1, 'Batch-size': 8, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('1.38039663600416956174e-05')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Xavier', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 8, 'Learning-rate': 0.1, 'Batch-size': 16, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('1.6390311163785510398e-05')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Xavier', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 8, 'Learning-rate': 0.1, 'Batch-size': 32, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('1.7421204348746734898e-05')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Xavier', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 8, 'Learning-rate': 0.1, 'Batch-size': 64, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('2.3518915147750944718e-05')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Xavier', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 8, 'Learning-rate': 0.5, 'Batch-size': 8, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('2.3240001531229691153e-06')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Xavier', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 8, 'Learning-rate': 0.5, 'Batch-size': 16, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('2.387489532554544803e-06')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Xavier', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 8, 'Learning-rate': 0.5, 'Batch-size': 32, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('2.5928053249512579752e-06')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Xavier', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 8, 'Learning-rate': 0.5, 'Batch-size': 64, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('3.846047399066998554e-06')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Xavier', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 16, 'Learning-rate': 0.01, 'Batch-size': 8, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.00035522663497663262835')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Xavier', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 16, 'Learning-rate': 0.01, 'Batch-size': 16, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.00018007022818475599133')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Xavier', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 16, 'Learning-rate': 0.01, 'Batch-size': 32, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.00017398307803881676548')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Xavier', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 16, 'Learning-rate': 0.01, 'Batch-size': 64, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.00020391282741360646183')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Xavier', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 16, 'Learning-rate': 0.1, 'Batch-size': 8, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('3.7640764341391022168e-05')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Xavier', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 16, 'Learning-rate': 0.1, 'Batch-size': 16, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('1.5617187281903709498e-05')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Xavier', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 16, 'Learning-rate': 0.1, 'Batch-size': 32, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('1.488358755454295021e-05')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Xavier', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 16, 'Learning-rate': 0.1, 'Batch-size': 64, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('1.8235303716727135464e-05')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Xavier', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 16, 'Learning-rate': 0.5, 'Batch-size': 8, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('7.3622194594049412085e-06')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Xavier', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 16, 'Learning-rate': 0.5, 'Batch-size': 16, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('2.7245506809351281104e-06')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Xavier', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 16, 'Learning-rate': 0.5, 'Batch-size': 32, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('2.6431288861902701631e-06')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Xavier', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 16, 'Learning-rate': 0.5, 'Batch-size': 64, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('1.7364129338543849465e-06')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Xavier', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 32, 'Learning-rate': 0.01, 'Batch-size': 8, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.00017161083543121939025')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Xavier', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 32, 'Learning-rate': 0.01, 'Batch-size': 16, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.00019476034392118298696')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Xavier', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 32, 'Learning-rate': 0.01, 'Batch-size': 32, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.00023381280725576427859')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Xavier', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 32, 'Learning-rate': 0.01, 'Batch-size': 64, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.00023467829516617453566')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Xavier', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 32, 'Learning-rate': 0.1, 'Batch-size': 8, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('1.3678233054959507649e-05')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Xavier', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 32, 'Learning-rate': 0.1, 'Batch-size': 16, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('1.572675431058466925e-05')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Xavier', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 32, 'Learning-rate': 0.1, 'Batch-size': 32, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('2.2930545631872820922e-05')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Xavier', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 32, 'Learning-rate': 0.1, 'Batch-size': 64, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('2.3694986538930876075e-05')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Xavier', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 32, 'Learning-rate': 0.5, 'Batch-size': 8, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('2.1271934778739112187e-06')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Xavier', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 32, 'Learning-rate': 0.5, 'Batch-size': 16, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('2.4324819767096420615e-06')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Xavier', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 32, 'Learning-rate': 0.5, 'Batch-size': 32, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('4.461200772235995358e-06')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Xavier', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 128, 'Learning-rate': 0.01, 'Batch-size': 8, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.00019264374175243780288')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Xavier', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 128, 'Learning-rate': 0.01, 'Batch-size': 16, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.00013888850081863820526')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Xavier', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 128, 'Learning-rate': 0.01, 'Batch-size': 32, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('5.2937134189826701565e-05')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Xavier', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 128, 'Learning-rate': 0.01, 'Batch-size': 64, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('4.2514162911679207232e-05')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Xavier', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 128, 'Learning-rate': 0.1, 'Batch-size': 8, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('2.3772676776067900226e-05')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Xavier', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 128, 'Learning-rate': 0.1, 'Batch-size': 16, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('1.5799168268341456443e-05')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Xavier', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 128, 'Learning-rate': 0.1, 'Batch-size': 32, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('3.4413517925910375276e-06')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Xavier', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 128, 'Learning-rate': 0.1, 'Batch-size': 64, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('2.9273968666385278604e-06')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Xavier', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 128, 'Learning-rate': 0.5, 'Batch-size': 8, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('4.798547566341991018e-06')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Xavier', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 128, 'Learning-rate': 0.5, 'Batch-size': 16, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('2.775920777956204615e-06')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Xavier', 'Normalization': 'Ninguna', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 2, 'Learning-rate': 0.01, 'Batch-size': 8, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.0008523900185247683851')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Xavier', 'Normalization': 'Ninguna', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 2, 'Learning-rate': 0.01, 'Batch-size': 16, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.0008523818218670548507')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Xavier', 'Normalization': 'Ninguna', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 2, 'Learning-rate': 0.01, 'Batch-size': 32, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.000852441123138408223')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Xavier', 'Normalization': 'Ninguna', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 2, 'Learning-rate': 0.01, 'Batch-size': 64, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.0008521615959621687315')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Xavier', 'Normalization': 'Ninguna', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 2, 'Learning-rate': 0.1, 'Batch-size': 8, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('7.476282497214433949e-05')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Xavier', 'Normalization': 'Ninguna', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 2, 'Learning-rate': 0.1, 'Batch-size': 16, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('7.476082376507228718e-05')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Xavier', 'Normalization': 'Ninguna', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 2, 'Learning-rate': 0.1, 'Batch-size': 32, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('7.4770527024237961455e-05')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Xavier', 'Normalization': 'Ninguna', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 2, 'Learning-rate': 0.1, 'Batch-size': 64, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('7.507446857800370546e-05')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Xavier', 'Normalization': 'Ninguna', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 2, 'Learning-rate': 0.5, 'Batch-size': 8, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('1.4488714735354250998e-05')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Xavier', 'Normalization': 'Ninguna', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 2, 'Learning-rate': 0.5, 'Batch-size': 16, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('1.45502156602702176046e-05')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Xavier', 'Normalization': 'Ninguna', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 2, 'Learning-rate': 0.5, 'Batch-size': 32, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('1.45958227630748502626e-05')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Xavier', 'Normalization': 'Ninguna', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 2, 'Learning-rate': 0.5, 'Batch-size': 64, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('2.098516725513166342e-05')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Xavier', 'Normalization': 'Ninguna', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 4, 'Learning-rate': 0.01, 'Batch-size': 8, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.00032352166580878246018')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Xavier', 'Normalization': 'Ninguna', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 4, 'Learning-rate': 0.01, 'Batch-size': 16, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.00032353504109426590172')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Xavier', 'Normalization': 'Ninguna', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 4, 'Learning-rate': 0.01, 'Batch-size': 32, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.00032373607709098778257')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Xavier', 'Normalization': 'Ninguna', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 4, 'Learning-rate': 0.01, 'Batch-size': 64, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.00032334082126514514228')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Xavier', 'Normalization': 'Ninguna', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 4, 'Learning-rate': 0.1, 'Batch-size': 8, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('2.1777899325076516859e-05')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Xavier', 'Normalization': 'Ninguna', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 4, 'Learning-rate': 0.1, 'Batch-size': 16, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('2.1796739736140430074e-05')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Xavier', 'Normalization': 'Ninguna', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 4, 'Learning-rate': 0.1, 'Batch-size': 32, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('2.1890521432235509655e-05')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Xavier', 'Normalization': 'Ninguna', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 4, 'Learning-rate': 0.1, 'Batch-size': 64, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('2.2502151737012196345e-05')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Xavier', 'Normalization': 'Ninguna', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 4, 'Learning-rate': 0.5, 'Batch-size': 8, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('4.521326542633785678e-06')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Xavier', 'Normalization': 'Ninguna', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 4, 'Learning-rate': 0.5, 'Batch-size': 16, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('6.580578624131385159e-06')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Xavier', 'Normalization': 'Ninguna', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 4, 'Learning-rate': 0.5, 'Batch-size': 32, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('1.4042810702732401783e-05')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Xavier', 'Normalization': 'Ninguna', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 4, 'Learning-rate': 0.5, 'Batch-size': 64, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('1.6075798867463569615e-05')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Xavier', 'Normalization': 'Ninguna', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 8, 'Learning-rate': 0.01, 'Batch-size': 8, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.00019453310118866960973')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Xavier', 'Normalization': 'Ninguna', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 8, 'Learning-rate': 0.01, 'Batch-size': 16, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.00019457010074224185339')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Xavier', 'Normalization': 'Ninguna', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 8, 'Learning-rate': 0.01, 'Batch-size': 32, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.0001941809192951972651')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Xavier', 'Normalization': 'Ninguna', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 8, 'Learning-rate': 0.01, 'Batch-size': 64, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.00019484127012220564514')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Xavier', 'Normalization': 'Ninguna', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 8, 'Learning-rate': 0.1, 'Batch-size': 8, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('1.5312062946702189768e-05')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Xavier', 'Normalization': 'Ninguna', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 8, 'Learning-rate': 0.1, 'Batch-size': 16, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('1.5700954601206401829e-05')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Xavier', 'Normalization': 'Ninguna', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 8, 'Learning-rate': 0.1, 'Batch-size': 32, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('1.4547237190156124437e-05')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Xavier', 'Normalization': 'Ninguna', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 8, 'Learning-rate': 0.1, 'Batch-size': 64, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('1.6238544394528492793e-05')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Xavier', 'Normalization': 'Ninguna', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 8, 'Learning-rate': 0.5, 'Batch-size': 8, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('3.0573318847743442093e-06')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Xavier', 'Normalization': 'Ninguna', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 8, 'Learning-rate': 0.5, 'Batch-size': 16, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('3.0357587144889810977e-06')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Xavier', 'Normalization': 'Ninguna', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 8, 'Learning-rate': 0.5, 'Batch-size': 32, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('6.563999566105839262e-06')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Xavier', 'Normalization': 'Ninguna', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 16, 'Learning-rate': 0.01, 'Batch-size': 8, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.0001023235256738695353')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Xavier', 'Normalization': 'Ninguna', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 16, 'Learning-rate': 0.01, 'Batch-size': 16, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.00010233168273627793217')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Xavier', 'Normalization': 'Ninguna', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 16, 'Learning-rate': 0.01, 'Batch-size': 32, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.00010292571535377140979')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Xavier', 'Normalization': 'Ninguna', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 16, 'Learning-rate': 0.01, 'Batch-size': 64, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.00010245644131280379747')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Xavier', 'Normalization': 'Ninguna', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 16, 'Learning-rate': 0.1, 'Batch-size': 8, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('1.1522794034377808664e-05')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Xavier', 'Normalization': 'Ninguna', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 16, 'Learning-rate': 0.1, 'Batch-size': 16, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('1.1429568656805459155e-05')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Xavier', 'Normalization': 'Ninguna', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 16, 'Learning-rate': 0.1, 'Batch-size': 32, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('1.0564308092192178072e-05')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Xavier', 'Normalization': 'Ninguna', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 16, 'Learning-rate': 0.1, 'Batch-size': 64, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('1.5588649222966303774e-05')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Xavier', 'Normalization': 'Ninguna', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 16, 'Learning-rate': 0.5, 'Batch-size': 8, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('2.6919500591521714156e-06')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Xavier', 'Normalization': 'Ninguna', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 16, 'Learning-rate': 0.5, 'Batch-size': 16, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('3.0298058487794893668e-06')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Xavier', 'Normalization': 'Ninguna', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 16, 'Learning-rate': 0.5, 'Batch-size': 32, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('1.9042233364225890451e-06')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Xavier', 'Normalization': 'Ninguna', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 32, 'Learning-rate': 0.01, 'Batch-size': 8, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('7.1055004782451301313e-05')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Xavier', 'Normalization': 'Ninguna', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 32, 'Learning-rate': 0.01, 'Batch-size': 16, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('7.10378744694065921e-05')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Xavier', 'Normalization': 'Ninguna', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 32, 'Learning-rate': 0.01, 'Batch-size': 32, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('7.1172556530497035135e-05')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Xavier', 'Normalization': 'Ninguna', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 32, 'Learning-rate': 0.01, 'Batch-size': 64, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('7.127632770733950725e-05')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Xavier', 'Normalization': 'Ninguna', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 32, 'Learning-rate': 0.1, 'Batch-size': 8, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('7.4928985946298742434e-06')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Xavier', 'Normalization': 'Ninguna', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 32, 'Learning-rate': 0.1, 'Batch-size': 16, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('7.057321138234737713e-06')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Xavier', 'Normalization': 'Ninguna', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 32, 'Learning-rate': 0.1, 'Batch-size': 32, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('8.043040172019289059e-06')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Xavier', 'Normalization': 'Ninguna', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 32, 'Learning-rate': 0.5, 'Batch-size': 8, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('1.9561344040412243252e-06')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Xavier', 'Normalization': 'Ninguna', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 128, 'Learning-rate': 0.01, 'Batch-size': 8, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('1.5298896310329361516e-05')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Xavier', 'Normalization': 'Ninguna', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 128, 'Learning-rate': 0.01, 'Batch-size': 16, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('1.5317704467565373988e-05')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Xavier', 'Normalization': 'Ninguna', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 128, 'Learning-rate': 0.01, 'Batch-size': 32, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('1.5200805848926771156e-05')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Xavier', 'Normalization': 'Ninguna', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 128, 'Learning-rate': 0.01, 'Batch-size': 64, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('1.3212263807989979896e-05')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Xavier', 'Normalization': 'Ninguna', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 128, 'Learning-rate': 0.1, 'Batch-size': 16, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('1.6153133421870108061e-06')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Normal distribution', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 2, 'Learning-rate': 0.01, 'Batch-size': 8, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.0015956271533923692635')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Normal distribution', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 2, 'Learning-rate': 0.01, 'Batch-size': 16, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.0014843304348064667737')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Normal distribution', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 2, 'Learning-rate': 0.01, 'Batch-size': 32, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.0010523500330941401403')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Normal distribution', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 2, 'Learning-rate': 0.01, 'Batch-size': 64, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.000998451437914716011')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Normal distribution', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 2, 'Learning-rate': 0.1, 'Batch-size': 8, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.00012830362140345525456')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Normal distribution', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 2, 'Learning-rate': 0.1, 'Batch-size': 16, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.00012547987126847158125')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Normal distribution', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 2, 'Learning-rate': 0.1, 'Batch-size': 32, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('7.165918917860192402e-05')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Normal distribution', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 2, 'Learning-rate': 0.1, 'Batch-size': 64, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('6.611586656633021292e-05')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Normal distribution', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 2, 'Learning-rate': 0.5, 'Batch-size': 8, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('2.2918922864308879919e-05')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Normal distribution', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 2, 'Learning-rate': 0.5, 'Batch-size': 16, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('2.3143617970402994835e-05')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Normal distribution', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 2, 'Learning-rate': 0.5, 'Batch-size': 32, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('1.2621734770528271305e-05')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Normal distribution', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 2, 'Learning-rate': 0.5, 'Batch-size': 64, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('1.1650502929549403912e-05')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Normal distribution', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 4, 'Learning-rate': 0.01, 'Batch-size': 8, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.001009169647356339128')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Normal distribution', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 4, 'Learning-rate': 0.01, 'Batch-size': 16, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.0009265340332464223681')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Normal distribution', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 4, 'Learning-rate': 0.01, 'Batch-size': 32, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.00052083040521237263225')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Normal distribution', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 4, 'Learning-rate': 0.01, 'Batch-size': 64, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.00046472199235289649655')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Normal distribution', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 4, 'Learning-rate': 0.1, 'Batch-size': 8, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('9.9250632024231245856e-05')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Normal distribution', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 4, 'Learning-rate': 0.1, 'Batch-size': 16, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('9.2957932614872807896e-05')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Normal distribution', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 4, 'Learning-rate': 0.1, 'Batch-size': 32, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('3.8627621376713074664e-05')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Normal distribution', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 4, 'Learning-rate': 0.1, 'Batch-size': 64, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('3.2289527981096669172e-05')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Normal distribution', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 4, 'Learning-rate': 0.5, 'Batch-size': 8, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('1.7602476700302859009e-05')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Normal distribution', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 4, 'Learning-rate': 0.5, 'Batch-size': 16, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('1.7154141401750651102e-05')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Normal distribution', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 4, 'Learning-rate': 0.5, 'Batch-size': 32, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('6.6762832973725442358e-06')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Normal distribution', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 4, 'Learning-rate': 0.5, 'Batch-size': 64, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('5.6513316445512198183e-06')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Normal distribution', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 8, 'Learning-rate': 0.01, 'Batch-size': 8, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.00024670675052776000088')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Normal distribution', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 8, 'Learning-rate': 0.01, 'Batch-size': 16, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.00031018298468523710968')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Normal distribution', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 8, 'Learning-rate': 0.01, 'Batch-size': 32, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.00031042537886732240224')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Normal distribution', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 8, 'Learning-rate': 0.01, 'Batch-size': 64, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.00036144420450455230034')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Normal distribution', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 8, 'Learning-rate': 0.1, 'Batch-size': 8, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('1.3353377842264161149e-05')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Normal distribution', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 8, 'Learning-rate': 0.1, 'Batch-size': 16, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('1.6003638549833652773e-05')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Normal distribution', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 8, 'Learning-rate': 0.1, 'Batch-size': 32, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('1.697134810807290818e-05')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Normal distribution', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 8, 'Learning-rate': 0.1, 'Batch-size': 64, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('2.2673466936531567442e-05')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Normal distribution', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 8, 'Learning-rate': 0.5, 'Batch-size': 8, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('2.233992309930230268e-06')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Normal distribution', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 8, 'Learning-rate': 0.5, 'Batch-size': 16, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('2.2578045056840116672e-06')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Normal distribution', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 8, 'Learning-rate': 0.5, 'Batch-size': 32, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('2.5498919130909718366e-06')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Normal distribution', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 8, 'Learning-rate': 0.5, 'Batch-size': 64, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('3.6916750239816337629e-06')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Normal distribution', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 16, 'Learning-rate': 0.01, 'Batch-size': 8, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.0005535213167779407275')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Normal distribution', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 16, 'Learning-rate': 0.01, 'Batch-size': 16, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.00028676451790134186306')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Normal distribution', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 16, 'Learning-rate': 0.01, 'Batch-size': 32, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.00028695502807498739605')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Normal distribution', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 16, 'Learning-rate': 0.01, 'Batch-size': 64, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.00033667518229497739506')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Normal distribution', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 16, 'Learning-rate': 0.1, 'Batch-size': 8, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('4.154033998037336209e-05')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Normal distribution', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 16, 'Learning-rate': 0.1, 'Batch-size': 16, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('1.6583454859409415952e-05')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Normal distribution', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 16, 'Learning-rate': 0.1, 'Batch-size': 32, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('1.7371154379797347147e-05')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Normal distribution', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 16, 'Learning-rate': 0.1, 'Batch-size': 64, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('2.1330135090405380866e-05')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Normal distribution', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 16, 'Learning-rate': 0.5, 'Batch-size': 8, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('7.194048705570651537e-06')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Normal distribution', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 16, 'Learning-rate': 0.5, 'Batch-size': 16, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('2.5966558206906267989e-06')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Normal distribution', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 16, 'Learning-rate': 0.5, 'Batch-size': 32, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('2.7517227844760766216e-06')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Normal distribution', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 16, 'Learning-rate': 0.5, 'Batch-size': 64, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('3.4695613251771361216e-06')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Normal distribution', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 32, 'Learning-rate': 0.01, 'Batch-size': 8, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.00033132914842554771328')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Normal distribution', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 32, 'Learning-rate': 0.01, 'Batch-size': 16, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.00036864445590723284172')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Normal distribution', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 32, 'Learning-rate': 0.01, 'Batch-size': 32, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.0004151539651575784686')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Normal distribution', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 32, 'Learning-rate': 0.01, 'Batch-size': 64, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.00041615616696134396765')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Normal distribution', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 32, 'Learning-rate': 0.1, 'Batch-size': 8, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('1.860951896962693118e-05')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Normal distribution', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 32, 'Learning-rate': 0.1, 'Batch-size': 16, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('2.2702178660936721033e-05')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Normal distribution', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 32, 'Learning-rate': 0.1, 'Batch-size': 32, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('3.2326387681592405013e-05')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Normal distribution', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 32, 'Learning-rate': 0.1, 'Batch-size': 64, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('3.3029861920850326347e-05')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Normal distribution', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 32, 'Learning-rate': 0.5, 'Batch-size': 8, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('2.5607559642806336164e-06')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Normal distribution', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 32, 'Learning-rate': 0.5, 'Batch-size': 16, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('3.1130141515558435235e-06')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Normal distribution', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 32, 'Learning-rate': 0.5, 'Batch-size': 32, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('5.11323297680104284e-06')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Normal distribution', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 32, 'Learning-rate': 0.5, 'Batch-size': 64, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('5.6018845359541952626e-06')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Normal distribution', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 128, 'Learning-rate': 0.01, 'Batch-size': 8, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.0011526310616550279791')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Normal distribution', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 128, 'Learning-rate': 0.01, 'Batch-size': 16, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.0008253200564551670252')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Normal distribution', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 128, 'Learning-rate': 0.01, 'Batch-size': 32, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.00035720629827542203702')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Normal distribution', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 128, 'Learning-rate': 0.01, 'Batch-size': 64, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.00029084057121396908735')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Normal distribution', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 128, 'Learning-rate': 0.1, 'Batch-size': 8, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.00011983297652315533086')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Normal distribution', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 128, 'Learning-rate': 0.1, 'Batch-size': 16, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('7.75719454871101939e-05')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Normal distribution', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 128, 'Learning-rate': 0.1, 'Batch-size': 32, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('2.4992373040236070721e-05')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Normal distribution', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 128, 'Learning-rate': 0.1, 'Batch-size': 64, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('1.8121123568197524525e-05')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Normal distribution', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 128, 'Learning-rate': 0.5, 'Batch-size': 8, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('2.2204298104335576416e-05')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Normal distribution', 'Normalization': 'Ninguna', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 2, 'Learning-rate': 0.01, 'Batch-size': 8, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.00069934520806422399765')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Normal distribution', 'Normalization': 'Ninguna', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 2, 'Learning-rate': 0.01, 'Batch-size': 16, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.0006992947312203682505')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Normal distribution', 'Normalization': 'Ninguna', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 2, 'Learning-rate': 0.01, 'Batch-size': 32, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.00070015075903069439885')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Normal distribution', 'Normalization': 'Ninguna', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 2, 'Learning-rate': 0.01, 'Batch-size': 64, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.0006989579206114209822')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Normal distribution', 'Normalization': 'Ninguna', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 2, 'Learning-rate': 0.1, 'Batch-size': 8, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('5.213211991693016157e-05')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Normal distribution', 'Normalization': 'Ninguna', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 2, 'Learning-rate': 0.1, 'Batch-size': 16, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('5.2075313409815106715e-05')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Normal distribution', 'Normalization': 'Ninguna', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 2, 'Learning-rate': 0.1, 'Batch-size': 32, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('5.209400741332321572e-05')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Normal distribution', 'Normalization': 'Ninguna', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 2, 'Learning-rate': 0.1, 'Batch-size': 64, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('5.1649485436710751524e-05')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Normal distribution', 'Normalization': 'Ninguna', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 2, 'Learning-rate': 0.5, 'Batch-size': 8, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('1.0477919762212937902e-05')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Normal distribution', 'Normalization': 'Ninguna', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 2, 'Learning-rate': 0.5, 'Batch-size': 16, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('1.6642386942378088544e-05')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Normal distribution', 'Normalization': 'Ninguna', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 2, 'Learning-rate': 0.5, 'Batch-size': 32, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('1.6512506815363202805e-05')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Normal distribution', 'Normalization': 'Ninguna', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 4, 'Learning-rate': 0.01, 'Batch-size': 8, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.0002524849818448590624')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Normal distribution', 'Normalization': 'Ninguna', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 4, 'Learning-rate': 0.01, 'Batch-size': 16, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.00025251526270161703402')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Normal distribution', 'Normalization': 'Ninguna', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 4, 'Learning-rate': 0.01, 'Batch-size': 32, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.00025312933829231331437')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Normal distribution', 'Normalization': 'Ninguna', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 4, 'Learning-rate': 0.01, 'Batch-size': 64, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.00025244763164737713207')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Normal distribution', 'Normalization': 'Ninguna', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 4, 'Learning-rate': 0.1, 'Batch-size': 8, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('2.0083211361482220921e-05')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Normal distribution', 'Normalization': 'Ninguna', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 4, 'Learning-rate': 0.1, 'Batch-size': 16, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('2.0114178335052868057e-05')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Normal distribution', 'Normalization': 'Ninguna', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 4, 'Learning-rate': 0.1, 'Batch-size': 32, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('2.0160934353536915407e-05')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Normal distribution', 'Normalization': 'Ninguna', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 4, 'Learning-rate': 0.1, 'Batch-size': 64, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('2.006608992383547523e-05')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Normal distribution', 'Normalization': 'Ninguna', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 4, 'Learning-rate': 0.5, 'Batch-size': 8, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('3.8696697192190204117e-06')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Normal distribution', 'Normalization': 'Ninguna', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 4, 'Learning-rate': 0.5, 'Batch-size': 16, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('3.7136740194373901301e-06')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Normal distribution', 'Normalization': 'Ninguna', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 4, 'Learning-rate': 0.5, 'Batch-size': 32, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('7.1065450991647428693e-06')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Normal distribution', 'Normalization': 'Ninguna', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 8, 'Learning-rate': 0.01, 'Batch-size': 8, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.00018029273478655291328')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Normal distribution', 'Normalization': 'Ninguna', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 8, 'Learning-rate': 0.01, 'Batch-size': 16, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.00018027117069797335664')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Normal distribution', 'Normalization': 'Ninguna', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 8, 'Learning-rate': 0.01, 'Batch-size': 32, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.00017968780260981871969')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Normal distribution', 'Normalization': 'Ninguna', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 8, 'Learning-rate': 0.01, 'Batch-size': 64, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.00018026858959331158667')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Normal distribution', 'Normalization': 'Ninguna', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 8, 'Learning-rate': 0.1, 'Batch-size': 8, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('1.42104641561382298995e-05')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Normal distribution', 'Normalization': 'Ninguna', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 8, 'Learning-rate': 0.1, 'Batch-size': 16, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('1.4213314251239489576e-05')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Normal distribution', 'Normalization': 'Ninguna', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 8, 'Learning-rate': 0.1, 'Batch-size': 32, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('1.4322769709622924437e-05')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Normal distribution', 'Normalization': 'Ninguna', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 8, 'Learning-rate': 0.1, 'Batch-size': 64, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('1.4429981481156636412e-05')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Normal distribution', 'Normalization': 'Ninguna', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 8, 'Learning-rate': 0.5, 'Batch-size': 8, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('2.6316849029875145404e-06')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Normal distribution', 'Normalization': 'Ninguna', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 8, 'Learning-rate': 0.5, 'Batch-size': 16, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('2.79983675616478214e-06')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Normal distribution', 'Normalization': 'Ninguna', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 8, 'Learning-rate': 0.5, 'Batch-size': 32, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('3.9262726043890078093e-06')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Normal distribution', 'Normalization': 'Ninguna', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 8, 'Learning-rate': 0.5, 'Batch-size': 64, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('1.7339057960924682863e-05')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Normal distribution', 'Normalization': 'Ninguna', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 16, 'Learning-rate': 0.01, 'Batch-size': 8, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.00016480103973768502409')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Normal distribution', 'Normalization': 'Ninguna', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 16, 'Learning-rate': 0.01, 'Batch-size': 16, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.00016478542391146996317')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Normal distribution', 'Normalization': 'Ninguna', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 16, 'Learning-rate': 0.01, 'Batch-size': 32, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.00016575698837830290432')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Normal distribution', 'Normalization': 'Ninguna', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 16, 'Learning-rate': 0.01, 'Batch-size': 64, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.00016478834294699583084')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Normal distribution', 'Normalization': 'Ninguna', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 16, 'Learning-rate': 0.1, 'Batch-size': 8, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('1.292261088672864777e-05')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Normal distribution', 'Normalization': 'Ninguna', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 16, 'Learning-rate': 0.1, 'Batch-size': 16, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('1.2920672960928351674e-05')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Normal distribution', 'Normalization': 'Ninguna', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 16, 'Learning-rate': 0.1, 'Batch-size': 32, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('1.3039184203161597719e-05')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Normal distribution', 'Normalization': 'Ninguna', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 16, 'Learning-rate': 0.1, 'Batch-size': 64, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('1.29934189442837737025e-05')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Normal distribution', 'Normalization': 'Ninguna', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 16, 'Learning-rate': 0.5, 'Batch-size': 8, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('2.3762317513562918456e-06')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Normal distribution', 'Normalization': 'Ninguna', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 16, 'Learning-rate': 0.5, 'Batch-size': 16, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('2.3773107421238525654e-06')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Normal distribution', 'Normalization': 'Ninguna', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 16, 'Learning-rate': 0.5, 'Batch-size': 32, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('2.7706004050823251926e-06')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Normal distribution', 'Normalization': 'Ninguna', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 16, 'Learning-rate': 0.5, 'Batch-size': 64, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('3.7049000412032389145e-05')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Normal distribution', 'Normalization': 'Ninguna', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 32, 'Learning-rate': 0.01, 'Batch-size': 8, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.00014541125272104426325')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Normal distribution', 'Normalization': 'Ninguna', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 32, 'Learning-rate': 0.01, 'Batch-size': 16, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.00014537028571719400525')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Normal distribution', 'Normalization': 'Ninguna', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 32, 'Learning-rate': 0.01, 'Batch-size': 32, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.00014544197777380893515')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Normal distribution', 'Normalization': 'Ninguna', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 32, 'Learning-rate': 0.01, 'Batch-size': 64, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.00014538329879176232474')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Normal distribution', 'Normalization': 'Ninguna', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 32, 'Learning-rate': 0.1, 'Batch-size': 8, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('1.1420094213393455561e-05')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Normal distribution', 'Normalization': 'Ninguna', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 32, 'Learning-rate': 0.1, 'Batch-size': 16, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('1.14118274078304182e-05')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Normal distribution', 'Normalization': 'Ninguna', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 32, 'Learning-rate': 0.1, 'Batch-size': 32, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('1.1417136533580370108e-05')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Normal distribution', 'Normalization': 'Ninguna', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 32, 'Learning-rate': 0.1, 'Batch-size': 64, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('1.13748107462965328525e-05')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Normal distribution', 'Normalization': 'Ninguna', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 32, 'Learning-rate': 0.5, 'Batch-size': 8, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('2.0461875656674395437e-06')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Normal distribution', 'Normalization': 'Ninguna', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 32, 'Learning-rate': 0.5, 'Batch-size': 16, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('2.0210398981354945833e-06')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Normal distribution', 'Normalization': 'Ninguna', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 32, 'Learning-rate': 0.5, 'Batch-size': 32, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('2.4253850938107446934e-06')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Normal distribution', 'Normalization': 'Ninguna', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 128, 'Learning-rate': 0.01, 'Batch-size': 8, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.00013725215574407708119')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Normal distribution', 'Normalization': 'Ninguna', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 128, 'Learning-rate': 0.01, 'Batch-size': 16, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.00013724978611145602372')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Normal distribution', 'Normalization': 'Ninguna', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 128, 'Learning-rate': 0.01, 'Batch-size': 32, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.0001364586387316006793')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Normal distribution', 'Normalization': 'Ninguna', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 128, 'Learning-rate': 0.01, 'Batch-size': 64, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.00013710615846221348103')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Normal distribution', 'Normalization': 'Ninguna', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 128, 'Learning-rate': 0.1, 'Batch-size': 8, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('1.0630274512987288666e-05')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Normal distribution', 'Normalization': 'Ninguna', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 128, 'Learning-rate': 0.1, 'Batch-size': 16, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('1.06382816889481699755e-05')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Normal distribution', 'Normalization': 'Ninguna', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 128, 'Learning-rate': 0.1, 'Batch-size': 32, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('1.0530458560334836288e-05')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Normal distribution', 'Normalization': 'Ninguna', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 128, 'Learning-rate': 0.1, 'Batch-size': 64, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('1.066484430458181888e-05')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Normal distribution', 'Normalization': 'Ninguna', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 128, 'Learning-rate': 0.5, 'Batch-size': 8, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('1.8373906526049394182e-06')}</li>
<li>{'Dataset': 'Iris', 'Initialization': 'Normal distribution', 'Normalization': 'Ninguna', 'Activation': 'Sigmoid', 'Input-size': 4, 'Output-size': 1, 'Hidden-layer-size': 128, 'Learning-rate': 0.5, 'Batch-size': 16, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('2.1798150021647170807e-06')}</li>
</ul>
</font>
<h2>Dataset Breast Cancer</h2>
<h3>Mejor accuracy: 0.95<br/>
Hay 8 clasificadores con el mismo accuracy</h3>
<font size='2'>
<ul>
<li>{'Dataset': 'Breast Cancer', 'Initialization': 'Xavier', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 30, 'Output-size': 1, 'Hidden-layer-size': 8, 'Learning-rate': 0.01, 'Batch-size': 8, 'Epochs': 2000, 'Accuracy': np.float64(0.956140350877193), 'Last-error': np.longdouble('0.025851965693099546158')}</li>
<li>{'Dataset': 'Breast Cancer', 'Initialization': 'Xavier', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 30, 'Output-size': 1, 'Hidden-layer-size': 32, 'Learning-rate': 0.01, 'Batch-size': 32, 'Epochs': 2000, 'Accuracy': np.float64(0.956140350877193), 'Last-error': np.longdouble('0.023814735872297627602')}</li>
<li>{'Dataset': 'Breast Cancer', 'Initialization': 'Xavier', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 30, 'Output-size': 1, 'Hidden-layer-size': 128, 'Learning-rate': 0.01, 'Batch-size': 32, 'Epochs': 2000, 'Accuracy': np.float64(0.956140350877193), 'Last-error': np.longdouble('0.024174934620899202784')}</li>
<li>{'Dataset': 'Breast Cancer', 'Initialization': 'Normal distribution', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 30, 'Output-size': 1, 'Hidden-layer-size': 4, 'Learning-rate': 0.01, 'Batch-size': 16, 'Epochs': 2000, 'Accuracy': np.float64(0.956140350877193), 'Last-error': np.longdouble('0.025434070957688489179')}</li>
<li>{'Dataset': 'Breast Cancer', 'Initialization': 'Normal distribution', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 30, 'Output-size': 1, 'Hidden-layer-size': 16, 'Learning-rate': 0.01, 'Batch-size': 16, 'Epochs': 2000, 'Accuracy': np.float64(0.956140350877193), 'Last-error': np.longdouble('0.029426853758799152185')}</li>
<li>{'Dataset': 'Breast Cancer', 'Initialization': 'Normal distribution', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 30, 'Output-size': 1, 'Hidden-layer-size': 32, 'Learning-rate': 0.01, 'Batch-size': 8, 'Epochs': 2000, 'Accuracy': np.float64(0.956140350877193), 'Last-error': np.longdouble('0.028253106502060502968')}</li>
<li>{'Dataset': 'Breast Cancer', 'Initialization': 'Normal distribution', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 30, 'Output-size': 1, 'Hidden-layer-size': 32, 'Learning-rate': 0.01, 'Batch-size': 16, 'Epochs': 2000, 'Accuracy': np.float64(0.956140350877193), 'Last-error': np.longdouble('0.027447667629961841704')}</li>
<li>{'Dataset': 'Breast Cancer', 'Initialization': 'Normal distribution', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 30, 'Output-size': 1, 'Hidden-layer-size': 128, 'Learning-rate': 0.01, 'Batch-size': 16, 'Epochs': 2000, 'Accuracy': np.float64(0.956140350877193), 'Last-error': np.longdouble('0.025773039791797582371')}</li>
</ul>
</font>
<h2>Dataset Wine</h2>
<h3>Mejor accuracy: 1.0<br/>
Hay 3 clasificadores con el mismo accuracy</h3>
<font size='2'>
<li>{'Dataset': 'Wine', 'Initialization': 'Xavier', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 13, 'Output-size': 1, 'Hidden-layer-size': 2, 'Learning-rate': 0.5, 'Batch-size': 32, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.025808646981718063858')}</li>
<li>{'Dataset': 'Wine', 'Initialization': 'Normal distribution', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 13, 'Output-size': 1, 'Hidden-layer-size': 2, 'Learning-rate': 0.5, 'Batch-size': 32, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.025658880823260419365')}</li>
<li>{'Dataset': 'Wine', 'Initialization': 'Normal distribution', 'Normalization': 'Z-score', 'Activation': 'Sigmoid', 'Input-size': 13, 'Output-size': 1, 'Hidden-layer-size': 16, 'Learning-rate': 0.5, 'Batch-size': 64, 'Epochs': 2000, 'Accuracy': np.float64(1.0), 'Last-error': np.longdouble('0.03513544955063513984')}</li>
</ul>
</font>


<h1>Graficación de los modelos</h1>

In [ ]:
def graficar(diccionario, mlp):
    """
    Grafica y guarda la imagen del error conforme a las épocas durante 
    el entrenamiento de la red neuronal
    """
    filename = f'{diccionario['Dataset']}_'
    filename += f'{diccionario['Initialization']}_'
    filename += f'{diccionario['Normalization']}_'
    filename += f'{diccionario['Hidden-layer-size']}_'
    filename += f'{int(diccionario['Learning-rate']*100)}pc_'
    filename += f'{diccionario['Batch-size']}.png'
    caracteristicas = f'Dataset: {diccionario['Dataset']}, '
    caracteristicas += f'Función de Inicialización: {diccionario['Initialization']}, '
    caracteristicas += f'Función de Normalización: {diccionario['Normalization']}, '
    caracteristicas += f'Neuronas ocultas: {diccionario['Hidden-layer-size']}, '
    caracteristicas += f'Learning Rate: {diccionario['Learning-rate']}, '
    caracteristicas += f'Batch Size: {diccionario['Batch-size']}'
    
    plt.figure(figsize=(16, 9))
    plt.title(f'Red Neuronal: {diccionario['Dataset']}')
    plt.figtext(0.13, 0.03, caracteristicas)
    plt.plot(np.arange(mlp.epochs), mlp.epochs_error, color='aqua', linestyle='-', linewidth=1, label='MSE')
    plt.xlabel('Época')
    plt.ylabel('MSE')
    plt.grid(True)
    plt.savefig(f'./graficas/{filename}')
    plt.close()
    
def graficar_conjunto(listas_modelos, nombres_labels, titulo, caracteristicas, filename, epocas, opacidad=0.2):
    """
    Grafica un conjunto de redes neuronales en la misma grafica
    """
    colores = ['black', 'darkred', 'darkviolet', 'forestgreen', 'darkorange', 'royalblue']
    marcadores = ['solid', (0, (3, 1, 1, 1)), 'dotted', (0, (5, 1)), 'dashed', (0, (5, 10)), (5, (10, 3)), (0, (3, 5, 1, 5))]
    i = 0
    offset = len(colores) - len(listas_modelos)
    patches = []
    plt.figure(figsize=(16, 9))
    plt.title(titulo)
    plt.figtext(0.13, 0.03, caracteristicas)
    for l in listas_modelos:    
        color = colores[i + offset]
        for t in l:
            plt.plot(np.arange(epocas), t[1].epochs_error[:epocas], color=color, linestyle=marcadores[i], linewidth=1, alpha=opacidad)
        patches.append(mpatches.Patch(color=color, label=nombres_labels[i]))
        i += 1
    plt.legend(handles=patches)
    plt.xlabel('Época')
    plt.ylabel('MSE')
    plt.grid(True)
    plt.savefig(filename)
    plt.close()

<h1>Crear y guardar graficas</h1>

In [ ]:
# --------------------------
# Graficar individualmente
# --------------------------
for d, c in lista_clasificadores:
    graficar(d, c)

In [ ]:
# --------------------------
# Graficar en conjunto
# --------------------------

# Filtro por cantidad de neuronas de la capa ocultas
Iris_neuronas = []
labels_Iris_neuronas = []
for n in neuronas_ocultas:
    Iris_neuronas.append([t for t in lista_clasificadores if t[0]['Dataset'] == 'Iris' 
                          and t[0]['Initialization'] == 'Normal distribution'
                          and t[0]['Normalization'] == 'Z-score' 
                          and t[0]['Hidden-layer-size'] == n])
    labels_Iris_neuronas.append(f'{n} neuronas ocultas')
graficar_conjunto(listas_modelos=Iris_neuronas,
                  nombres_labels=labels_Iris_neuronas,
                  titulo='Dataset Iris', 
                  caracteristicas='Dataset: Iris, Inicialización: Distribución normal, Normalización: Z-score', 
                  filename='Iris_neuronas.png', 
                  epocas=300,
                  opacidad=0.5)

# Filtro por learning rate
Iris_learning_rate = []
labels_Iris_learning_rate = []
for lr in learning_rate:
    Iris_learning_rate.append([t for t in lista_clasificadores if t[0]['Dataset'] == 'Iris' 
                               and t[0]['Initialization'] == 'Normal distribution'
                               and t[0]['Learning-rate'] == lr])
    labels_Iris_learning_rate.append(f'Learning rate: {lr}')
graficar_conjunto(listas_modelos=Iris_learning_rate,
                  nombres_labels=labels_Iris_learning_rate,
                  titulo='Dataset Iris', 
                  caracteristicas='Dataset: Iris, Inicialización: Distribución normal',
                  filename='Iris_learning_rate.png', 
                  epocas=300,
                  opacidad=0.4)

# Filtro por tamaño de batch del dataset Cáncer
cancer_batch = []
labels_cancer_batch = []
for b in batch_size:
    cancer_batch.append([t for t in lista_clasificadores if t[0]['Dataset'] == 'Breast Cancer' 
                      and t[0]['Initialization'] == 'Normal distribution'
                      and t[0]['Normalization'] == 'Z-score' 
                      and t[0]['Learning-rate'] == 0.1
                      and t[0]['Batch-size'] == b])
    labels_cancer_batch.append(f'Tamaño de batch: {b}')
graficar_conjunto(listas_modelos=cancer_batch,
                  nombres_labels=labels_cancer_batch,
                  titulo='Dataset Cáncer de mama', 
                  caracteristicas='Dataset: Cáncer de mama, Inicialización: Distribución normal, Normalización: Z-score, Learning rate 0.1', 
                  filename='Cancer_batch.png',  
                  epocas=100,
                  opacidad=0.85)

# Filtro por forma de inicialización
vino_init = []
labels_vino_init = []
for init in nombres_inicializacion:
    vino_init.append([t for t in lista_clasificadores if t[0]['Dataset'] == 'Wine'
                      and t[0]['Initialization'] == init
                      and t[0]['Normalization'] == 'Z-score' 
                      and t[0]['Learning-rate'] == 0.01])
    labels_vino_init.append(f'Inicialización: {init}')
graficar_conjunto(listas_modelos=vino_init,
                  nombres_labels=labels_vino_init,
                  titulo='Dataset Vino', 
                  caracteristicas='Dataset: Vino, Normalización: Z-score, Learning rate 0.01', 
                  filename='Vino_init.png',  
                  epocas=200,
                  opacidad=0.5)

# Filtro por forma de normalización para el dataset de Cáncer de mama
cancer_normalizacion = []
labels_cancer_normalizacion = []
for norm in nombres_normalizacion:
    cancer_normalizacion.append([t for t in lista_clasificadores if t[0]['Dataset'] == 'Breast Cancer'
                                 and t[0]['Normalization'] == norm])
    labels_cancer_normalizacion.append(f'Normalización: {norm}')
graficar_conjunto(listas_modelos=cancer_normalizacion,
                  nombres_labels=labels_cancer_normalizacion,
                  titulo='Dataset Cáncer de mama', 
                  caracteristicas='Dataset: Cáncer de mama', 
                  filename='Cancer_norm.png',  
                  epocas=200)

# Filtro por forma de normalización para el dataset de Vino
wine_normalizacion = []
labels_wine_normalizacion = []
for norm in nombres_normalizacion:
    wine_normalizacion.append([t for t in lista_clasificadores if t[0]['Dataset'] == 'Wine'
                                 and t[0]['Normalization'] == norm])
    labels_wine_normalizacion.append(f'Normalización: {norm}')
graficar_conjunto(listas_modelos=wine_normalizacion,
                  nombres_labels=labels_wine_normalizacion,
                  titulo='Dataset Vino', 
                  caracteristicas='Dataset: Vino', 
                  filename='Vino_norm.png',  
                  epocas=200)

# Filtro por forma de normalización para el dataset de Iris
iris_normalizacion = []
labels_iris_normalizacion = []
for norm in nombres_normalizacion:
    iris_normalizacion.append([t for t in lista_clasificadores if t[0]['Dataset'] == 'Iris'
                                 and t[0]['Normalization'] == norm])
    labels_iris_normalizacion.append(f'Normalización: {norm}')
graficar_conjunto(listas_modelos=iris_normalizacion,
                  nombres_labels=labels_iris_normalizacion,
                  titulo='Dataset Iris', 
                  caracteristicas='Dataset: Iris', 
                  filename='Iris_norm.png',  
                  epocas=200)

# Filtro por forma de normalización para el dataset de XOR
xor_normalizacion = []
labels_xor_normalizacion = []
for norm in nombres_normalizacion:
    xor_normalizacion.append([t for t in lista_clasificadores if t[0]['Dataset'] == 'XOR'
                                 and t[0]['Normalization'] == norm])
    labels_xor_normalizacion.append(f'Normalización: {norm}')
graficar_conjunto(listas_modelos=xor_normalizacion,
                  nombres_labels=labels_xor_normalizacion,
                  titulo='Dataset XOR', 
                  caracteristicas='Dataset: XOR', 
                  filename='XOR_norm.png',
                  epocas=200)